In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopy.distance
import geopy.point


In [2]:
df_skiers = pd.read_csv("../data/filtered/Dati_filtrati_all.csv")
df_skiers = df_skiers[df_skiers["DNF"] == False]

In [3]:
dist_df = pd.DataFrame(columns=["Athlete", "Day", "File_gps", "Distance"])
for day in ["G1", "P1", "P2"]:
    df_skiers_day = df_skiers[df_skiers["Day"] == day]

    for athlete, skier_file, startTime, duration in zip(df_skiers_day["Atleta"], df_skiers_day["File_gps"], df_skiers_day["StartTime"], df_skiers_day["Time"]):
        df_skier = pd.read_csv(skier_file)
        df_skier_started = df_skier[df_skier["Timestamp"] >= startTime]
        df_skier_run = df_skier_started[df_skier_started["Timestamp"] <= startTime + duration]
        df_skier_run = df_skier_run.sort_values('Timestamp', ascending=False)
        distance = 0
        for long1, lat1, alt1, long2, lat2, alt2 in zip(df_skier_run["GNSS_Longitude"][:-1], df_skier_run["GNSS_Latitude"][:-1], df_skier_run["GNSS_Altitude"][:-1],df_skier_run["GNSS_Longitude"][1:], df_skier_run["GNSS_Latitude"][1:],df_skier_run["GNSS_Altitude"][1:]):

            distance2d = geopy.distance.geodesic((long1, lat1), (long2, lat2)).m
            distance += np.sqrt(distance2d**2 + (alt1 - alt2)**2)

        dist_df.loc[len(dist_df)] = [athlete, day, skier_file, distance]
dist_df.to_csv("../data/filtered/distances.csv", index = False)